In [1]:
import psycopg2
import pandas as pd
import nltk
import spacy 
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter
from nltk.corpus import stopwords
import string
import numpy as np
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from src import galgraphs
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bhaskarballapragada/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bhaskarballapragada/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def retrieve_reviews(product):
    conn = psycopg2.connect(dbname='product_reviews', user='postgres', password='', host='localhost')
    cursor = conn.cursor()
    p_no = 0
    r_no = 0
    cursor.execute("select * from reviews where r_comments like 'Verified%' product_name = '{}'".format(product))
    rows = cursor.fetchall()
    df = pd.DataFrame(rows)
    cursor.close()
    conn.close()
    return df

def retrieve_all_reviews():
    conn = psycopg2.connect(dbname='product_reviews', user='postgres', password='', host='localhost')
    cursor = conn.cursor()
    p_no = 0
    r_no = 0
    cursor.execute("select * from reviews where r_comments like 'Verified%' ")
    rows = cursor.fetchall()
    df = pd.DataFrame(rows)
    cursor.close()
    conn.close()
    return df


In [3]:
df = retrieve_all_reviews()
columns = ['id','p_name','url','p_no','r_no','r_stars','r_date','r_name','r_title','r_text','r_comments']
df.columns = columns

df_new = df.copy()
df_new = df_new[df_new['r_stars'].apply(lambda x: x in ['5.0 out of 5 stars','1.0 out of 5 stars'])]
df_new['rating'] = df_new['r_stars'].apply(lambda x: 1 if x == '5.0 out of 5 stars' else 0)
df_new = df_new[['rating','p_name','r_name','r_title','r_text']]

df_nb = df_new[['rating','p_name']].copy()
df_nb.columns = ['rating','product']
df_nb['reviews'] = df['r_title'] + ' ' + df['r_text']
df_nb.head(3)

,rating,product,reviews
0,1,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,"TV Great, TruMotion Bad! Now that I found the ..."
1,1,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,Hint: Turn OFF Smart Picture Mode Hint: Turn o...
2,1,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,43UJ6300 Needed an upgrade for my 7yr old HDTV...


In [13]:
# from nltk.corpus import stopwords
# from nltk.stem.wordnet import WordNetLemmatizer
# import string
# stopwords = set(stopwords.words('english'))
# punctuation = set(string.punctuation)
# lemmatize = WordNetLemmatizer()

# def cleaning(article):
#     one = " ".join([i for i in article.lower().split() if i not in stopwords])
#     two = "".join(i for i in one if i not in punctuation)
#     three = " ".join(lemmatize.lemmatize(i) for i in two.split())
#     return three

# df_text = df_new[['r_text']]
# df_text.info()

# data = df_text.applymap(cleaning)['r_text']
# text = data
# text_list = [i.split() for i in text]
# len(text_list)

In [12]:
from sklearn.model_selection import train_test_split
df_new['des'] = df_new['r_title'] + " " + df_new['r_text']
df_new.head(2)
X_train, X_test, y_train, y_test = train_test_split(df_new['des'], df_new['rating'], train_size=0.8)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [14]:
nlp = spacy.load("en")

def pos_neg_sentiments(df_product):
    #positive sentiments
    g = df_product.groupby('p_name')
    for name, group in g:
        group_pos = group[group['rating'] == 1]
        group_neg = group[group['rating'] == 0]
        
        if ((group_pos.shape[0] > 20) and (group_neg.shape[0] > 20)):
            
            tfidf_pos = TfidfVectorizer(stop_words='english',max_features=10000)

            X_pos_descr_vectors = tfidf_pos.fit_transform(group_pos['des'])
            nb_pos = MultinomialNB()
            nb_pos.fit(X_pos_descr_vectors, group_pos['rating'].transpose()) 
            y_hat_pos = nb_pos.predict_proba(X_pos_descr_vectors)
            arr = np.argsort(nb_pos.feature_log_prob_[0])[-20:-1]
            list_of_pos_words = []
            for i in arr:
                list_of_pos_words.append(tfidf_pos.get_feature_names()[i])
                
            tfidf_neg = TfidfVectorizer(stop_words='english',max_features=10000)            
            X_neg_descr_vectors = tfidf_neg.fit_transform(group_neg['des'])
            nb_neg = MultinomialNB()
            nb_neg.fit(X_neg_descr_vectors, group_neg['rating'].transpose()) 
            y_hat_neg = nb_neg.predict_proba(X_neg_descr_vectors)
            arr = np.argsort(nb_neg.feature_log_prob_[0])[-20:-1]
                
            list_of_neg_words = []
            for i in arr:
                list_of_neg_words.append(tfidf_neg.get_feature_names()[i])

            print (name, '\n')    
            print ("Positive Words \n\t", list_of_pos_words)
            print ("\nNegative Words \n\t", list_of_neg_words)
            print ("______________________________________________________________________________")
            tokens = nlp(' '.join(list_of_pos_words))
            #for token1 in tokens:
                #for token2 in tokens:
                    #print(token1.text, token2.text, token1.similarity(token2))
            
            
            

print (pos_neg_sentiments(df_new))

/anaconda3/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


LG Electronics 65UJ6300 65-Inch 4K Ultra HD Smart LED TV (2017 Model) 

Positive Words 
	 ['excellent', 'value', 'use', 'amazing', 'clear', 'perfect', 'lg', 'easy', 'nice', 'sound', 'works', 'quality', 'awesome', 'price', 'good', 'love', 'picture', 'tv', 'great']

Negative Words 
	 ['shattered', 'disappointed', 'working', 'damaged', 'arrival', 'defective', 'item', 'product', 'picture', 'opened', 'days', 'cracked', 'return', 'came', 'arrived', 'box', 'broken', 'screen', 'tv']
______________________________________________________________________________
Samsung Electronics UN65MU6300 65-Inch 4K Ultra HD Smart LED TV (2017 Model) 

Positive Words 
	 ['beautiful', 'works', 'happy', 'smart', 'amazing', 'awesome', 'sound', 'nice', 'samsung', 'set', 'quality', 'easy', 'price', 'excellent', 'good', 'love', 'picture', 'great', 'tv']

Negative Words 
	 ['bought', 'customer', 'received', 'dont', 'cracked', 'problem', 'service', 'box', 'return', 'just', 'work', 'broken', 'buy', 'picture', 'workin

In [4]:
from src.naive_review_analyzer import NaiveReviewAnalyzer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bhaskarballapragada/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bhaskarballapragada/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
nra = NaiveReviewAnalyzer(df_nb)

In [6]:
nra.create_bow()

In [8]:
for name, group in df_nb.groupby('product'):
    print(name, '\n\t',nra.create_word_list(product=name, rating=0))

/anaconda3/lib/python3.6/site-packages/sklearn/naive_bayes.py:461: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


LG Electronics 65UJ6300 65-Inch 4K Ultra HD Smart LED TV (2017 Model) 
	 [('shattered', 'VERB'), ('disappointed', 'ADJ'), ('working', 'VERB'), ('damaged', 'VERB'), ('arrival', 'NOUN'), ('defective', 'ADJ'), ('item', 'NOUN'), ('product', 'NOUN'), ('picture', 'NOUN'), ('opened', 'VERB'), ('days', 'NOUN'), ('cracked', 'VERB'), ('return', 'NOUN'), ('came', 'VERB'), ('arrived', 'VERB'), ('box', 'NOUN'), ('broken', 'VERB'), ('screen', 'NOUN'), ('tv', 'NOUN')]
LG Electronics 65UJ7700 65-Inch 4K Ultra HD Smart LED TV (2017 Model) 
	 [('switched', 'VERB'), ('best', 'ADJ'), ('gaming', 'NOUN'), ('watch', 'NOUN'), ('quality', 'NOUN'), ('box', 'NOUN'), ('blurry', 'VERB'), ('bought', 'VERB'), ('credit', 'NOUN'), ('screen', 'NOUN'), ('star', 'NOUN'), ('great', 'ADJ'), ('buy', 'NOUN'), ('received', 'VERB'), ('damaged', 'VERB'), ('returned', 'VERB'), ('sony', 'ADJ'), ('picture', 'NOUN'), ('lg', 'INTJ')]
Samsung Electronics QN65Q7C Curved 65-Inch 4K Ultra HD Smart QLED TV (2017 Model) 
	 [('times', 'NOU